<a href="https://colab.research.google.com/github/mhuertascompany/DL_ED127_2022/blob/main/hands-on/CNNs/SED_AGE_ED127_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Galaxy Star Formation History with deep learning

The goal of this notebook is to develop a deep learning model which takes a galaxy Spectral Energy Distribution (SED) and estimates some parameters of the history of star formation (SFH), i.e. the star formation rate as a function of comsic time. 

We will use to that purpose data from the hydrodinamic cosmological simulation [EAGLE.](https://eagle.strw.leidenuniv.nl/), for which we can extract the Star Formation Histories. We then take galaxies at z=0 and generate mock SEDs based on their stellar populations. 

The goal is therefore to establish a mapping between the SED and the SFH. The SFH is parametrized with some moments, t_xx measuring the time at which xx of the stellar mass of the galaxy has been formed. That way the problem is converted into a regression. 


In [ ]:
#%pylab inline
!pip install -q tfds-nightly
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds


data_dir ='/content/drive/MyDrive/ED127_2022/EAGLE'

## Mount Drive

Before mounting the drive click on [this folder](https://drive.google.com/drive/folders/1PcftgBzBySo1Ync-Wdsp9arTCJ_MfEPE?usp=sharing) and add it to your google drive by following these steps:

*   Go to your drive 
*   Find shared folder ("Shared with me" link)
*   Right click it
*   "Add Shortcut to Drive"



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load Tensorflow dataset

In [ ]:
import sys
sys.path.insert(1, "/content/drive/MyDrive/ED127_2022/EAGLE")

In [ ]:
import eagle

The next cell loads the dataset. We use here Tensorflow Dataset which is a convenient way of generating the data for training. There are predefined datasets. In our case, the code to build the dataset is in the file /content/drive/MyDrive/ED127_2022/EAGLE/eagle.py

In [ ]:
dset = tfds.load('eagle', split='train', data_dir=data_dir)

Visualizes some trainng data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
print("Train",len(dset))

fig, axs = plt.subplots(1, 1)
for example in dset.take(1):
    #print((example['wl_sort'])[example['inds_valid']])
    time_vec = example['time']
    inds_valid = example['inds_valid']
    axs.scatter((example['wl_sort'])[example['inds_valid']],np.log10(example['sed']))
    axs.set_xscale('log')
    plt.xlabel("Wavelength",fontsize=20)
    plt.ylabel("Flux",fontsize=20)

fig, axs = plt.subplots(1, 1)
for example in dset.take(1):
    #print((example['wl_sort'])[example['inds_valid']])
    time_vec = example['time']
    axs.scatter(time_vec,example["Mstar"])
    axs.set_yscale('log')
    plt.xlabel("Normalized Cosmic Time",fontsize=20)
    plt.ylabel("Stellar Mass",fontsize=20)
    #mass_history_summaries = find_summaries(np.flip(example['Mstar']),
                                                #np.flip(example['time']))
    #print(mass_history_summaries)
    for m in example["mass_quantiles"]:
      plt.axvline(x=m,color='red')  

## Training

Generates the functions for training. 

In [ ]:
# this function outputs the t50 value (time at which half of the mass has been formed)
# can be modified
def preprocessing(example):
    sed = -1*tf.experimental.numpy.log10(example['sed'])/2
    t50 = (example['mass_quantiles'])[:,4]/100
    return sed, t50

def input_fn(mode='train', batch_size=64, 
             dataset_name='tng100', data_dir=None,
             include_mass=True, arctan=True):
    """
    mode: 'train' or 'test'
    """
    keys = ['sed','mass_quantiles']
    if mode == 'train':
        dataset = tfds.load(dataset_name, split='train[:80%]', data_dir=data_dir)
        dataset = dataset.map(lambda x: {k:x[k] for k in keys})
        dataset = dataset.repeat()
        dataset = dataset.shuffle(10000)
    elif mode=='val':
        dataset = tfds.load(dataset_name, split='train[80%:90%]', data_dir=data_dir)
        dataset = dataset.map(lambda x: {k:x[k] for k in keys}) 
    else:
        dataset = tfds.load(dataset_name, split='train[90%:]', data_dir=data_dir)
        dataset = dataset.map(lambda x: {k:x[k] for k in keys})
        
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.map(preprocessing)
    dataset = dataset.prefetch(-1)       # fetch next batches while training current one (-1 for autotune)
    return dataset

Creates the datasets for training, validation and test

In [ ]:
batch_size = 64
epochs = 20

dtrain = input_fn(mode='train', batch_size=batch_size, dataset_name='eagle',data_dir=data_dir)
dval = input_fn(mode='val', batch_size=batch_size, dataset_name='eagle',data_dir=data_dir)
dtest = input_fn(mode='test', batch_size=batch_size, dataset_name='eagle',data_dir=data_dir)

Creates the model

In [ ]:
tfd = tfp.distributions
tfb = tfp.bijectors
tfkl = tf.keras.layers

num_components = 1
event_shape = [1]

params_size = tfp.layers.MixtureSameFamily.params_size(
    num_components,
    component_params_size=tfp.layers.IndependentNormal.params_size(event_shape))

sed_net = tf.keras.Sequential([
        tfkl.Input(shape=(125, 1)),
        
        #ADD 1D convolutions here

        tfkl.Flatten(),
        tfkl.Dense(128, activation='relu'),
        tfkl.Dropout(0.4),
        tfkl.Dense(8, activation='softplus'),
        tfkl.Dense(units=params_size, activation=None),
        tfp.layers.MixtureSameFamily(num_components, tfp.layers.IndependentNormal(event_shape))
])

negloglik = lambda y, p_y: -p_y.log_prob(y)


opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
sed_net.compile(loss=negloglik, optimizer=opt)

In [ ]:
sed_net.summary()

Training loop

In [ ]:
hist = sed_net.fit(dtrain, 
                     epochs=epochs,
                     steps_per_epoch=1000,validation_data=dval)

## Evaluation of results

Gets the mean and standard deviations of the posteriors estimated by the neural network

In [ ]:
time_est_avg = np.concatenate([sed_net(batch[0]).mean() for batch in dtest])
time_est_std = np.concatenate([sed_net(batch[0]).stddev() for batch in dtest])

Gets the ground truth

In [ ]:
total = []
for element in dtest.as_numpy_iterator(): 
  total.append(element[1])

time_true = np.concatenate(total)  

Plots the results

In [ ]:
plt.errorbar(time_true,time_est_avg,yerr=time_est_std,color='black',fmt="none")
plt.scatter(time_true,time_est_avg,color='red',s=2)
plt.plot(np.linspace(0,1,100),np.linspace(0,1,100),color='red')

## Exercices

### Exercice 1: 
Create a model that predicts several times simultaneously.

### Exercice 2:

Create a model that uses only the optical/NIR part of the SED as input, i.e. wl < 20.000 A


### Exercice 3

Add 10% error to the SED and estimate the impact on the results